Less of a unit test, more an exploration of waffles.

In [98]:
filename = 'samples/waffle.tex'

In [99]:
# debug
from importlib import reload
import TexSoup
reload(TexSoup)

import TexSoup
import re
import numpy as np

from TexSoup import preprocessing, postprocess

In [100]:
with open(filename,'r') as f:
    tex_doc = f.read()

Sometimes there are weird errors like spaces in begins/ends -- fix these first:

In [101]:
# from TexSoup import preprocessing
reload(preprocessing)

<module 'TexSoup.preprocessing' from '/Users/jnaiman/TexSoup/TexSoup/preprocessing.py'>

In [102]:
tex_doc = preprocessing.process_begin_end(tex_doc)

Clean accents in case folks did something funny:

In [103]:
tex_doc, error_accents = preprocessing.clean_accents(tex_doc)

In [104]:
#print(tex_doc)

Get new commands and enviroments -- we need to these to check if there are `\begin` and `\end` in these, so first, get them!

In [105]:
newcommands, newenvironments = preprocessing.get_newcommands_and_newenvs(tex_doc,
                                                                         verbose=True)

Get the argument numbers for these -- will need this for search-replace:

In [106]:
args_newcommands = preprocessing.find_args_newcommands(newcommands,error_out=True, verbose=True)

In [107]:
args_newcommands

[['\\be', '\\newcommand{\\be}{\\begin{equation}}', 57, 91, 0],
 ['\\ee', '\\newcommand{\\ee}{\\end{equation}}', 92, 124, 0],
 ['\\bee',
  '\\newcommand{\\bee}[1]{\\begin{equation}\\label{e:#1}}',
  126,
  176,
  1],
 ['\\eee', '\\newcommand{\\eee}{\\end{equation}}', 177, 210, 0],
 ['\\beee',
  '\\newcommand{\\beee}[2]{\\begin{equation}\\label{e:#1} #2}',
  212,
  266,
  2],
 ['\\eeee', '\\newcommand{\\eeee}{\\end{equation}}', 267, 301, 0],
 ['\\etal ', '\\def \\etal {{\\em et al.}}', 304, 329, 0]]

In [108]:
args_newenvironments = preprocessing.find_args_newenvironments(newenvironments,
                                                               error_out=True, verbose=True)

In [109]:
args_newenvironments

[]

In [110]:
comments, find_replace, error = preprocessing.generate_find_replace_newcommands(args_newcommands, 
                                                               verbose=True,arg_type = 'newcommand')

Replace newcommands and environments:

In [111]:
tex_doc_nc, error, warnings = preprocessing.replace_newcommands_and_newenvironments(tex_doc, 
                                                                          args_newcommands,
                                                                          args_newenvironments,
                                                                          verbose=True)

args for: \newcommand{\bee}[1]{\begin{equation}\label{e:#1}}
{1: 'lenseq'}
args for: \newcommand{\beee}[2]{\begin{equation}\label{e:#1} #2}
{1: 'lenseq2', 2: 'A(x)=5'}

\newcommand{\be}{\begin{equation}} gets commented
\newcommand{\ee}{\end{equation}} gets commented
\newcommand{\bee}[1]{\begin{equation}\label{e:#1}} gets commented
\newcommand{\eee}{\end{equation}} gets commented
\newcommand{\beee}[2]{\begin{equation}\label{e:#1} #2} gets commented
\newcommand{\eeee}{\end{equation}} gets commented


In [112]:
soup = TexSoup.TexSoup(r''+tex_doc_nc, tolerance=0)

In [113]:
reload(TexSoup)
import TexSoup.postprocess as postprocess
reload(postprocess)

<module 'TexSoup.postprocess' from '/Users/jnaiman/TexSoup/TexSoup/postprocess.py'>

In [114]:
soup_clean = postprocess.clean_slash_commands(soup)

In [115]:
for iss,s in enumerate(soup_clean.all):
    print('----------- soup element =', iss, '---------------')
    if type(s.expr) == TexSoup.data.TexNamedEnv: # parts of the text
        if 'begin' in s.expr.begin and 'document' in s.expr.begin:
            for isss,ss in enumerate(s.all):
                print('----------- sub-soup element =', isss, '---------------')
                #print(type(ss))
                print(type(ss.expr))
                print(ss)
                print('-----------------')
                if str(ss).strip() == '\\': # 
                    import sys; sys.exit()
        else:
            print(' ******** not in document *********** ')

----------- soup element = 0 ---------------
----------- soup element = 1 ---------------
----------- soup element = 2 ---------------
----------- soup element = 3 ---------------
----------- soup element = 4 ---------------
----------- soup element = 5 ---------------
----------- soup element = 6 ---------------
----------- soup element = 7 ---------------
----------- soup element = 8 ---------------
----------- soup element = 9 ---------------
----------- soup element = 10 ---------------
----------- soup element = 11 ---------------
----------- soup element = 12 ---------------
----------- soup element = 13 ---------------
----------- soup element = 14 ---------------
----------- soup element = 15 ---------------
----------- soup element = 16 ---------------
----------- soup element = 17 ---------------
----------- soup element = 18 ---------------
----------- soup element = 19 ---------------
----------- soup element = 20 ---------------
----------- soup element = 21 --------------

In [121]:
from TexSoup import postprocess
reload(TexSoup)
reload(postprocess)
reload(preprocessing)

#from TexSoup.preprocessing import accents, accents_alone
from TexSoup.postprocess import parse_soup

In [122]:
#get_replacement_tex = TexSoup.postprocess.get_replacement_tex

In [123]:
# verbose = False
# tex_doc_accent = tex_doc_nc


# icount = 0
# texout_arr = []
# errorAll = False
# for isss, s in enumerate(soup.all):
#     if type(s.expr) == TexSoup.data.TexNamedEnv: # parts of the text
#         if 'begin' in s.expr.begin and 'document' in s.expr.begin:
#             # add preamble
#             #texout_arr.append((preamble,'preamble'))
#             texout_arr.append(('\\begin{document}\n','beginDoc'))
#             try: 
#                 iterator = s.all
#             except:
#                 errorAll = True
#             if not errorAll:
#                 for is3,ss in enumerate(s.all):  
#                     if type(ss.expr) == TexSoup.data.TexText: # mark text words
#                         strout = get_replacement_tex(tex_doc_accent, s, is3,ss, verbose=verbose)
#                         if len(strout.strip()) > 0: # not just white-space
#                             if strout.lstrip()[0] != '%': # not a comment:
#                                 if str(ss.expr) in accents_alone: # accent alone
#                                     strout = get_replacement_tex(tex_doc_accent, s, is3,ss, verbose=verbose)
#                                     texout_arr.append((strout, 'accent'))
#                                 elif texout_arr[-1][0] in accents:
#                                     strout = get_replacement_tex(tex_doc_accent, s, is3,ss, verbose=verbose)
#                                     texout_arr[-1] = ((texout_arr[-1][0] + strout, 'accent')) # put together
#                                 else:
#                                     texout_arr.append((strout, 'text'))
#                             else: # for all comments
#                                 texout_arr.append((strout, 'comment'))
#                         else: # just whitespace
#                             texout_arr.append((strout,'whitespace'))
#                     elif 'cite' in ss.name: # citations
#                         strout = get_replacement_tex(tex_doc_accent, s, is3,ss, verbose=verbose)
#                         texout_arr.append((strout, 'citation'))
#                     elif 'ref' in ss.name: # references
#                         strout = get_replacement_tex(tex_doc_accent, s, is3,ss, verbose=verbose)
#                         texout_arr.append((strout, 'reference'))
#                     elif type(ss.expr) == TexSoup.data.TexMathModeEnv: # inline math
#                         strout = get_replacement_tex(tex_doc_accent, s, is3,ss, verbose=verbose)
#                         texout_arr.append((strout,'inline'))
#                     elif type(ss.expr) == TexSoup.data.BraceGroup:
#                         strout = get_replacement_tex(tex_doc_accent, s, is3,ss, verbose=verbose)
#                         isAccent = False
#                         for a in accents:
#                             if a in strout:
#                                 isAccent = True
#                         if not isAccent:
#                             for a in accents_alone:
#                                 if a in strout:
#                                     isAccent = True
#                         if isAccent:
#                             texout_arr.append((strout,'accent'))
#                         else:
#                             # possible that last one is an accent
#                             if texout_arr[-1][0] in accents:
#                                 texout_arr[-1] = ((texout_arr[-1][0] + strout, 'accent')) # put together
#                             else: # just a bracket
#                                 texout_arr.append((strout,'bracket'))
#                     else: # a command or named environment
#                         # special ones
#                         if str(ss.expr).strip() == '\\S':
#                             strout = get_replacement_tex(tex_doc_accent, s, is3,ss, verbose=verbose)
#                             texout_arr.append((strout, 'S-command'))
#                         elif type(ss.expr) == TexSoup.data.TexNamedEnv:
#                             strout = get_replacement_tex(tex_doc_accent, s, is3,ss, verbose=verbose)
#                             #if 'eqnarray' in str(ss): import sys; sys.exit()
#                             texout_arr.append((strout, 'namedEnv'))
#                         elif str(ss.expr) in accents_alone: # accent alone
#                             strout = get_replacement_tex(tex_doc_accent, s, is3,ss, verbose=verbose)
#                             texout_arr.append((strout, 'accent'))
#                         elif texout_arr[-1][0] in accents:
#                             strout = get_replacement_tex(tex_doc_accent, s, is3,ss, verbose=verbose)
#                             texout_arr[-1] = ((texout_arr[-1][0] + strout, 'accent')) # put together
#                             #import sys; sys.exit()
#                         else:
#                             strout = get_replacement_tex(tex_doc_accent, s, is3,ss, verbose=verbose)
#                             texout_arr.append((strout,'commandOrBracket'))

#             else:
#                 continue
#             texout_arr.append(('\\end{document}\n','endDoc'))
#         else: # something else -- outside of document
#             print('not in begin/end!', s)
#             #import sys; sys.exit()
#             ##errorAll = True
#             strout = get_replacement_tex(tex_doc_accent, soup, isss,s, verbose=verbose)
#             texout_arr.append((strout,'outside'))
#     else: # typically \usepackage commands, \documentclass, etc
#         strout = get_replacement_tex(tex_doc_accent, soup, isss,s, verbose=verbose)
#         texout_arr.append((strout, 'others'))


In [125]:
texout_arr = parse_soup(soup,tex_doc_nc,verbose=False)

In [127]:
for t,ttype in texout_arr:
    print('------', ttype, '-----')
    print(t)

------ others -----
\documentclass[a4paper]{article}
------ others -----


------ others -----


------ others -----
\usepackage{hyperref}
------ others -----


------ others -----


------ others -----
%\newcommand{\be}{\begin{equation}}
------ others -----


------ others -----
%\newcommand{\ee}{\end{equation}}
------ others -----


------ others -----


------ others -----
%\newcommand{\bee}[1]{\begin{equation}\label{e:#1}}
------ others -----


------ others -----
%\newcommand{\eee}{\end{equation}}
------ others -----


------ others -----


------ others -----
%\newcommand{\beee}[2]{\begin{equation}\label{e:#1} #2}
------ others -----


------ others -----
%\newcommand{\eeee}{\end{equation}}
------ others -----


------ others -----


------ others -----


------ others -----
\def \etal {{\em et al.}}
------ others -----


------ others -----


------ beginDoc -----
\begin{document}

------ whitespace -----


------ whitespace -----


------ commandOrBracket -----
\section{Waffles

In [25]:
texout_arr[-5:]

[('\\`{g}', 'accent'),
 (' it.\n\n', 'text'),
 ("\\'Ok great!\n\nWell that was fun!\n\n\n", 'accent'),
 ('\\end{document}\n', 'endDoc'),
 ('\n', 'others')]

In [26]:
from string import whitespace

In [90]:
# combine accents
texout_arr2 = []
text = ''
for it,(t,ttype) in enumerate(texout_arr):
    if ttype == 'accent':
        # what is the ender of the last one?
        if texout_arr2[-1][0][-1] not in whitespace: # yup, connect the two together
            texout_arr2[-1] = ((texout_arr2[-1][0]+t,'textWithAccent'))
            #import sys; sys.exit()
        else: # just leave as is
            texout_arr2.append((t,'textWithAccent'))
    else:
        texout_arr2.append((t,ttype))
        
# also combine before... 
# should probably do this in one loop for efficiency but... you know how it goes
texout_arr3 = []
skips = 0
for it,(t,ttype) in enumerate(texout_arr2):
    if skips==0:
        #if it>66: import sys; sys.exit()
        if ttype == 'textWithAccent': #check after
            ik = 1
            if it+ik < len(texout_arr2)-1: # not at end
                c = texout_arr2[it+ik][0]
                cout = ''
                #import sys; sys.exit()
                #print('b:', c)
                #print('b:', cout)
                if c[0] not in whitespace:
                    #print('here')
                    #import sys; sys.exit()
                    while c[0] not in whitespace and it+ik < len(texout_arr2)-1:
                        c = texout_arr2[it+ik][0]
                        #print(c)
                        cout += c
                        #print(cout)
                        ik += 1
                        #print('also here')
                    #print('now here')
                    #import sys; sys.exit()
                    # if texout_arr2[it+1][0][0] not in whitespace: # have non-whitespace
                    #     texout_arr3.append((t+texout_arr2[it+1][0],'textWithAccent'))
                    #     skip=True
                    #     print(texout_arr3[-1])
                    #     import sys; sys.exit()
                    #import sys; sys.exit()
                    texout_arr3.append((t+cout, 'textWithAccent'))
                    skips = ik-1
                    #if skips != 3: import sys; sys.exit()
                    #import sys; sys.exit()
                else: # have whitespace, should be it's own thing
                    texout_arr3.append((t,ttype))
            else:
                texout_arr3.append((t,ttype))
        else:
            texout_arr3.append((t,ttype))
    else:
        #print(skips, ":", t)
        #print(texout_arr3[-1])
        skips -= 1

In [92]:
# for t,ttype in texout_arr3:
#     print('----------',ttype,'----------')
#     print(t)